<img src='./img/DataStore_EUMETSAT_Copernicus.png'/>

Copyright (c) 2024 EUMETSAT <br>
License: MIT

<hr>

<a href="./index.ipynb">← Index</a>
<br>
<a href="./1_3_Downloading_products.ipynb">← Downloading products</a><span style="float:right;"><a href="./1_5_MTG_FCI_data_access.ipynb">Downloading and visualising MTG FCI products →</a>

# Sentinel-3 Data Store access with EUMDAC

### Data used

| Product Description  | Data Store collection ID| Product Navigator |
|:--------------------:|:-----------------------:|:-----------------:|
| Sentinel-3 OLCI level-2 full resolution  | EO:EUM:DAT:0407 | [link](https://navigator.eumetsat.int/product/EO:EUM:DAT:SENTINEL-3:OL_2_WFR___NTC?query=OLCI&filter=satellite__Sentinel-3&filter=instrument__OLCI&filter=processingLevel__Level%202%20Data&s=advanced) |

### Learning outcomes

At the end of this notebook you will know;
* How to refine your <font color="#138D75">**searches**</font> for Sentinel-3 products in the EUMETSAT Data Store using the `eumdac` API client
* How to <font color="#138D75">**download**</font> components of products
* How to pre-screen downloads based on flags

### Outline

Data from Sentinel-3 is available through multiple sources, either via a web user interface (WebUI) or through code and command line interfaces with an Application Programming Interface (API). WebUIs are useful for accessing quick-look data visualisations, and for browsing to see what is available. APIs are more useful for routine, automated and operational data access. Here we will guide you through ways you can access Sentinel-3 data through both methods. We will use OLCI Level-2 Full Resolution data as an example.

1. [Download data via GUIs](#section1)
1. [Authenticating the APIs](#section2)
1. [Example 1: Filter by collections](#section3)
1. [Example 2: Filter by time](#section4)
1. [Example 3: Filter by space and time](#section5)
1. [Example 4: Download by component](#section6)

<hr>

In [2]:
import os
import json
import datetime
import shutil
import eumdac
import requests
from IPython.display import YouTubeVideo, HTML

# Create a download directory for our downloaded products
download_dir = os.path.join(os.getcwd(), "products")
os.makedirs(download_dir, exist_ok=True)

## Downloading data via GUIs

### The EUMETSAT Data Store

The [Data Store](https://data.eumetsat.int) is EUMETSAT's primary pull service for delivering data, including the ocean data available from Sentinel-3. 

Access to it is possible through a WebUI, and through a series of application programming interfaces (APIs). The Data Store supports browsing, searching and downloading data as well as subscription services. It also provides a link to the online version of the [EUMETSAT Data Tailor](https://tailor.eumetsat.int/) for customisation. The video below provides an overview of the Data Store WebUI.

The video below explains how you can access data through the EUMETSAT Data Store WebUI. You can visit and see if you can find Sentinel-3 data to download, however this notebook will also show you how to access the Data Store API, facilitated by the `eumdac` client (see the [Downloading via the Data Store API](#section3) section, below).
#### Links:

* [EUMETSAT Data Store](https://data.eumetsat.int)
* [More information on the Data Store](https://user.eumetsat.int/data-access/data-store)

<div class="alert alert-info" role="alert">

## <a id='section1'></a>Authenticating the APIs
[Back to top](#TOC_TOP)

</div>

To access Sentinel-3 data from the [Data Store](https://data.eumetsat.int), we will use the EUMETSAT Data Access Client (`eumdac`). If you are working with the recommended Anaconda Python distribution, you can install eumdac using;

`conda install -c eumetsat eumdac`

or with pip using;

`pip install eumdac`


However, you can also find the source code on the [EUMETSAT gitlab](https://gitlab.eumetsat.int/eumetlab/data-services/eumdac). Please visit the EUMETSAT user portal spaces for the the [Data Store](https://user.eumetsat.int/data-access/data-store) and [eumdac](https://user.eumetsat.int/resources/user-guides/eumetsat-data-access-client-eumdac-guide) for more information.

In order to allow us to download data from the Data Store via API, we need to provide our credentials. We can do this in two ways; either by creating a file called `.eumdac_credentials` in our home directory (*option 1 - recommended*) or by supplying our credentials directly in this script (*option 2*). 

#### Option 1: creating  `.eumdac_credentials` in our home directory

For most computer systems the home directory can be found at the path \user\username, /users/username, or /home/username depending on your operating system.

In this file we need to add the following information exactly as follows;

```
{
"consumer_key": "<your_consumer_key>",
"consumer_secret": "<your_consumer_secret>"
}
```

You must replace `<your_consumer_key>` and `<your_consumer_secret>` with the information you extract from https://api.eumetsat.int/api-key/. You will need a [EUMETSAT User Portal account](https://user.eumetsat.int/) to access this link, and in order to see the information you must click the "Show hidden fields" button at the bottom of the page.

*Note: your key and secret are permanent, so you only need to do this once, but you should take care to never share them*

Once you have done this, you can read in your credentials using the commands in the following cell. These will be used to generate a time-limited token, which will refresh itself when it expires.

In [ ]:
# load credentials
with open(os.path.join(os.path.expanduser("~"),'.eumdac_credentials')) as json_file:
    credentials = json.load(json_file)
    token = eumdac.AccessToken((credentials['consumer_key'], credentials['consumer_secret']))

    
try:
    print(f"This token '{token}' expires {token.expiration}")
except requests.exceptions.HTTPError as exc:
    print(f"Error when tryng the request to the server: '{exc}'")

#### Option 2: provide credentials directly

You can provide your credentials directly as follows;

`token = eumdac.AccessToken((consumer_key, consumer_secret))`

Once we have a token (by either method described above) we can create a datastore 'object' that contains all the collections, and filter this list for those that are relevant to Sentinel-3. The code to the right of the colon is what will tell use the ID for the Sentinel-3 collection we are interested in.

*Note: this method is convenient in the short term, but is not really recommended as you have to put your key and secret in this notebook, and run the risk of accidentally sharing them. This method also requires you to authenticate on a notebook-by-notebook basis.*


In [ ]:
# insert your personal key and secret into the single quotes

consumer_key = 'YOUR_CONSUMER_KEY'
consumer_secret = 'YOUR_CONSUMER_SECRET'

credentials = (consumer_key, consumer_secret)
token = eumdac.AccessToken(credentials) 

try:
    print(f"This token '{token}' expires {token.expiration}")
except requests.exceptions.HTTPError as exc:
    print(f"Error when tryng the request to the server: '{exc}'")

After you set your credentials (as described in Option 1 or Option 2) you can connect to the service and looking for all Sentinel-3 collection in EUMETSATs DataStore:

In [ ]:
# create data store object
datastore = eumdac.DataStore(token)

# show all collections:
for collectionID in datastore.collections:
    if 'Sentinel-3' in collectionID.title:
        try:
            print(f"{collectionID}: {collectionID.title}")
        except eumdac.datastore.DataStoreError as error:
            print(f"Error related to the data store: '{error.msg}'")
        except eumdac.collection.CollectionError as error:
            print(f"Error related to a collection: '{error.msg}'")
        except requests.exceptions.RequestException as error:
            print(f"Unexpected error: {error}")

<div class="alert alert-block alert-success">
<b>NOTE:</b><br />
Find more information about EUMDAC errors, their causes and possible solutions, in our knowledge base: <a href="https://user.eumetsat.int/resources/user-guides/eumetsat-data-access-client-eumdac-guide#ID-Exception-handling">https://user.eumetsat.int/resources/user-guides/eumetsat-data-access-client-eumdac-guide#ID-Exception-handling</a>
</div>

Now we can select our collection of interest, which in this case is EO:EUM:DAT:0407 for OLCI Level 2 Ocean Colour Full Resolution.

<div class="alert alert-info" role="alert">

## <a id='section1'></a>Example 1: Filtering by collection
[Back to top](#TOC_TOP)

</div>

Now that we know out collection of interest, we can use `eumdac` to create collection object

In [10]:
# set collection ID for OLCI L2 WFR
collectionID = 'EO:EUM:DAT:0407'

# Use collection ID
selected_collection = datastore.get_collection(collectionID)
try:
    print(selected_collection.title)
except eumdac.collection.CollectionError as error:
    print(f"Error related to a collection: '{error.msg}'")
except requests.exceptions.RequestException as error:
    print(f"Unexpected error: {error}")

OLCI Level 2 Ocean Colour Full Resolution - Sentinel-3


<div class="alert alert-info" role="alert">

## <a id='section2'></a>Example 2: Filtering by time
[Back to top](#TOC_TOP)

</div>

Now that we specified our collection, we can query the collections it contains. For example, to find the lastest product in a collection;

In [12]:
# Get the latest product in a collection
selected_collection = datastore.get_collection(collectionID)
latest = selected_collection.search().first()
    
try:
    print(latest)
except eumdac.collection.CollectionError as error:
    print(f"Error related to the collection: '{error.msg}'")
except requests.exceptions.RequestException as error:
    print(f"Unexpected error: {error}")

S3A_OL_2_WFR____20230313T080518_20230313T080808_20230313T104312_0169_096_263_4140_MAR_O_NR_003.SEN3


Alternatively, we can search for and list all the products that occur within a time range.

In [13]:
# time filter the collection for products
start = datetime.datetime(2022, 3, 23, 16, 0)
end = datetime.datetime(2022, 3, 23, 16, 15)
products = selected_collection.search(dtstart=start, dtend=end)
    
for product in products:    
    try:
        print(product)
    except eumdac.collection.CollectionError as error:
        print(f"Error related to the collection: '{error.msg}'")
    except eumdac.product.ProductError as error:
            print(f"Error related to the product: '{error.msg}'")
    except requests.exceptions.RequestException as error:
        print(f"Unexpected error: {error}")

S3A_OL_2_WFR____20220323T161304_20220323T161604_20220325T023952_0179_083_211_2880_MAR_O_NT_003.SEN3
S3A_OL_2_WFR____20220323T161004_20220323T161304_20220325T023931_0179_083_211_2700_MAR_O_NT_003.SEN3
S3A_OL_2_WFR____20220323T160704_20220323T161004_20220325T023909_0179_083_211_2520_MAR_O_NT_003.SEN3
S3A_OL_2_WFR____20220323T160404_20220323T160704_20220325T023849_0179_083_211_2340_MAR_O_NT_003.SEN3
S3A_OL_2_WFR____20220323T160104_20220323T160404_20220325T023829_0179_083_211_2160_MAR_O_NT_003.SEN3
S3A_OL_2_WFR____20220323T155804_20220323T160104_20220325T023809_0179_083_211_1980_MAR_O_NT_003.SEN3


<div class="alert alert-info" role="alert">

## <a id='section3'></a>Example 3: Filtering by space and time
[Back to top](#TOC_TOP)

</div>

Alongside temporal filtering, we can also pass "Well Known Text (WKT)" polygons to `eumdac` to specify a region of interest (ROI) that we want to search over.

In [14]:
# space/time filter the collection for products
selected_collection = datastore.get_collection(collectionID)
start = datetime.datetime(2022, 1, 23)
end = datetime.datetime(2022, 1, 24)
WKT = "POLYGON((-1.0 -1.0,4.0 -4.0,8.0 -2.0,9.0 2.0,6.0 4.0,1.0 5.0,-1.0 -1.0))"

products = selected_collection.search(
    geo=WKT,
    dtstart=start, 
    dtend=end)

In [15]:
for product in products:
    try:
        print(product)
    except eumdac.collection.CollectionError as error:
        print(f"Error related to the collection: '{error.msg}'")
    except eumdac.product.ProductError as error:
        print(f"Error related to the product: '{error.msg}'")
    except requests.exceptions.RequestException as error:
        print(f"Unexpected error: {error}")

S3A_OL_2_WFR____20220123T100205_20220123T100505_20220124T223346_0179_081_136_3060_MAR_O_NT_003.SEN3
S3A_OL_2_WFR____20220123T095905_20220123T100205_20220124T223325_0179_081_136_2880_MAR_O_NT_003.SEN3
S3B_OL_2_WFR____20220123T092235_20220123T092535_20220124T223420_0179_061_378_3060_MAR_O_NT_003.SEN3
S3B_OL_2_WFR____20220123T091935_20220123T092235_20220124T223353_0179_061_378_2880_MAR_O_NT_003.SEN3


<div class="alert alert-info" role="alert">

## <a id='section4'></a>Example 4: Downloading entire products
[Back to top](#TOC_TOP)

</div>

Once we have identified a product, we can download it simply by passing the product and collectionID to the `get_product` method of our `datastore` object.

In [16]:
selected_product = datastore.get_product(product_id=str(product), collection_id=collectionID)

try:
    with selected_product.open() as fsrc, open(os.path.join(download_dir, fsrc.name), mode='wb') as fdst:
        print(f'Downloading {fsrc.name}')
        shutil.copyfileobj(fsrc, fdst)
        print(f'Download of product {fsrc.name} finished.')
except eumdac.product.ProductError as error:
        print(f"Error related to the product: '{error.msg}'")
except requests.exceptions.RequestException as error:
    print(f"Unexpected error: {error}")

Download of product S3B_OL_2_WFR____20220123T091935_20220123T092235_20220124T223353_0179_061_378_2880_MAR_O_NT_003.SEN3.zip finished.


<div class="alert alert-info" role="alert">

## <a id='section4'></a>Example 5: Downloading by component
[Back to top](#TOC_TOP)

</div>

If we do not wish to download an entire product, we can choose to select only a single component using the `entries` property of the product.

In [17]:
# Get the latest product in a collection
selected_collection = datastore.get_collection(collectionID)
latest = selected_collection.search().first()

for entry in latest.entries:
    if 'xfdumanifest.xml' in entry:
        try:
            with latest.open(entry=entry) as fsrc, open(os.path.join(os.getcwd(), 'products',fsrc.name), mode='wb') as fdst:
                print(f'Downloading {fsrc.name}.')
                shutil.copyfileobj(fsrc, fdst)
                print(f'Download of file {fsrc.name} finished.')
        except eumdac.collection.CollectionError as error:
            print(f"Error related to the collection: '{error.msg}'")
        except eumdac.product.ProductError as error:
                print(f"Error related to the product: '{error.msg}'")
        except requests.exceptions.RequestException as error:
            print(f"Unexpected error: {error}")

Download of file xfdumanifest.xml finished.


<a href="./index.ipynb">← Index</a>
<br>
<a href="./1_3_Downloading_products.ipynb">← Downloading products</a><span style="float:right;"><a href="./1_5_MTG_data_access.ipynb">Downloading and visualising MTG FCI products →</a>

<hr>

<p style="text-align:left;">This project is licensed under the <a href="./LICENSE.TXT">MIT License</a> <span style="float:right;"><a href="https://gitlab.eumetsat.int/eumetlab/data-services/eumdac_data_store">View on GitLab</a> | <a href="https://classroom.eumetsat.int/">EUMETSAT Training</a> | <a href=mailto:ops@eumetsat.int>Contact</a></span></p>